# **Convolutional neural network architecture for geometric matching**

**Authors: Ignacio Rocco  (IDI), Relja Arandjelovic (ENS), Josef Sivic (3CIIRC)**

**Official Github**: https://github.com/ignacio-rocco/cnngeometric_pytorch

---

**Edited By Su Hyung Choi (Key Summary & Code Practice)**

If you have any issues on this scripts, please PR to the repository below.

**[Github: @JonyChoi - Computer Vision Paper Reviews]** https://github.com/jonychoi/Computer-Vision-Paper-Reviews

Edited Jan 10 2022

---

### **Abstract**

<i>
                    We address the problem of determining correspondences
                    between two images in agreement with a geometric model
                    such as an affine or thin-plate spline transformation, and
                    estimating its parameters. The contributions of this work
                    are three-fold. First, we propose a convolutional neural network architecture for geometric matching. The architecture
                    is based on three main components that mimic the standard
                    steps of feature extraction, matching and simultaneous inlier detection and model parameter estimation, while being
                    trainable end-to-end. Second, we demonstrate that the network parameters can be trained from synthetically generated imagery without the need for manual annotation and
                    that our matching layer significantly increases generalization capabilities to never seen before images. Finally, we
                    show that the same model can perform both instance-level
                    and category-level matching giving state-of-the-art results
                    on the challenging Proposal Flow dataset.
                </i>

### **1. Introduction**

<p>
                    Estimating correspondences between images is one of
                    the fundamental problems in computer vision [19, 25] with
                    applications ranging from large-scale 3D reconstruction [3]
                    to image manipulation [21] and semantic segmentation
                    [42]. Traditionally, correspondences consistent with a geometric model such as epipolar geometry or planar affine
                    transformation, are computed by detecting and matching
                    local features (such as SIFT [38] or HOG [12, 22]), followed by pruning incorrect matches using local geometric
                    constraints [43, 47] and robust estimation of a global geometric transformation using algorithms such as RANSAC
                    [18] or Hough transform [32, 34, 38]. This approach works
                    well in many cases but fails in situations that exhibit (i) large
                    changes of depicted appearance due to e.g. intra-class variation [22], or (ii) large changes of scene layout or non-rigid deformations that require complex geometric models with
                    many parameters which are hard to estimate in a manner
                    robust to outliers.
                </p>
                <table>
                    <tbody>
                        <tr>
                            <td>
                                <img src="./imgs/figure1.png" width="550px" />
                            </td>
                            <td>
                                Figure 1: Our trained geometry estimation network automatically
                                aligns two images with substantial appearance differences. It is
                                able to estimate large deformable transformations robustly in the
                                presence of clutter.
                            </td>
                        </tr>
                    </tbody>
                </table>
                <p>
                    In this work we build on the traditional approach and
                    develop a convolutional neural network (CNN) architecture
                    that mimics the standard matching process. First, we replace the standard local features with powerful trainable
                    convolutional neural network features [31, 46], which allows us to handle large changes of appearance between
                    the matched images. Second, we develop trainable matching and transformation estimation layers that can cope with
                    noisy and incorrect matches in a robust way, mimicking the
                    good practices in feature matching such as the second nearest neighbor test [38], neighborhood consensus [43, 47] and
                    Hough transform-like estimation [32, 34, 38].
                </p>
                <p>
                    The outcome is a convolutional neural network architecture trainable for the end task of geometric matching,
                    which can handle large appearance changes, and is therefore
                    suitable for both instance-level and category-level matching
                    problems.
</p>


### **2. Related Work**

<p>            
                    The classical approach for finding correspondences involves identifying interest points and computing local descriptors around these points [10, 11, 24, 37, 38, 39, 43].
                    6148
                    While this approach performs relatively well for instancelevel matching, the feature detectors and descriptors lack
                    the generalization ability for category-level matching.
                </p>
                <p>
                    Recently, convolutional neural networks have been used
                    to learn powerful feature descriptors which are more robust
                    to appearance changes than the classical descriptors [9, 23,
                    28, 45, 52]. However, these works still divide the image into
                    a set of local patches and extract a descriptor individually
                    from each patch. Extracted descriptors are then compared
                    with an appropriate distance measure [9, 28, 45], by directly
                    outputting a similarity score [23, 52], or even by directly
                    outputting a binary matching/non-matching decision [4].
                </p>
                <p>
                    In this work, we take a different approach, treating the
                    image as a whole, instead of a set of patches. Our approach
                    has the advantage of capturing the interaction of the different parts of the image in a greater extent, which is not possible when the image is divided into a set of local regions.
                </p>
                <p>
                    Related are also network architectures for estimating
                    inter-frame motion in video [17, 48, 50] or instance-level
                    homography estimation [14], however their goal is very different from ours, targeting high-precision correspondence
                    with very limited appearance variation and background
                    clutter. Closer to us is the network architecture of [29]
                    which, however, tackles a different problem of fine-grained
                    category-level matching (different species of birds) with
                    limited background clutter and small translations and scale
                    changes, as their objects are largely centered in the image.
                    In addition, their architecture is based on a different matching layer, which we show not to perform as well as the
                    matching layer used in our work.
                </p>
                <p>
                    Some works, such as [11, 15, 22, 30, 35, 36], have addressed the hard problem of category-level matching, but
                    rely on traditional non-trainable optimization for matching
                    [11, 15, 30, 35, 36], or guide the matching using object proposals [22]. On the contrary, our approach is fully trainable
                    in an end-to-end manner and does not require any optimization procedure at evaluation time, or guidance by object proposals.
                </p>
                <p>
                    Others [33, 44, 53] have addressed the problems of instance and category-level correspondence by performing
                    joint image alignment. However, these methods differ from
                    ours as they: (i) require class labels; (ii) don’t use CNN features; (iii) jointly align a large set of images, while we align
                    image pairs; and (iv) don’t use a trainable CNN architecture
                    for alignment as we do.
</p>


### **3. Architecture for geometric matching**
<p>
                    In this section, we introduce a new convolutional neural network architecture for estimating parameters of a geometric transformation between two input images. The architecture is designed to mimic the classical computer vision pipeline (e.g. [40]), while using differentiable modules
                    so that it is trainable end-to-end for the geometry estimation task. The classical approach consists of the following
                    stages: (i) local descriptors (e.g. SIFT) are extracted from
                    both input images, (ii) the descriptors are matched across
                    images to form a set of tentative correspondences, which
                    are then used to (iii) robustly estimate the parameters of the
                    geometric model using RANSAC or Hough voting.
                </p>     
                <table>
    <tbody>
        <tr>
            <td>
                            <img src="./imgs/figure2.png" width="500px" />
            </td>
            <td>
                            <img src="./imgs/figure2_description.png" width="500px" />
            </td>
        </tr>
    </tbody>
</table>
                <p>
                    Our architecture, illustrated in Fig. 2, mimics this process by: (i) passing input images IA and IB through a
                    siamese architecture consisting of convolutional layers, thus
                    extracting feature maps fA and fB which are analogous to
                    dense local descriptors, (ii) matching the feature maps (“descriptors”) across images into a tentative correspondence
                    map fAB, followed by a (iii) regression network which directly outputs the parameters of the geometric model, ˆθ, in
                    a robust manner. The inputs to the network are the two images, and the outputs are the parameters of the chosen geometric model, e.g. a 6-D vector for an affine transformation.
                </p>
                <p>
                    In the following, we describe each of the three stages in detail.
                </p>

### **3.1. Feature extraction**

<p>
                    The first stage of the pipeline is feature extraction, for
                    which we use a standard CNN architecture. A CNN without fully connected layers takes an input image and produces a feature map f ∈ R
                    h×w×d
                    , which can be interpreted
                    as a h × w dense spatial grid of d-dimensional local descriptors. A similar interpretation has been used previously
                    in instance retrieval [5, 7, 8, 20] demonstrating high discriminative power of CNN-based descriptors. Thus, for feature extraction we use the VGG-16 network [46], cropped
                    at the pool4 layer (before the ReLU unit), followed by
                    per-feature L2-normalization. We use a pre-trained model,
                    originally trained on ImageNet [13] for the task of image
                    classification. As shown in Fig. 2, the feature extraction network is duplicated and arranged in a siamese configuration
                    such that the two input images are passed through two identical networks which share parameters.
                </p>

### **3.2. Matching network**
<p>
                    The image features produced by the feature extraction
                    networks should be combined into a single tensor as input to
                    the regressor network to estimate the geometric transformation. We first describe the classical approach for generating
                    tentative correspondences, and then present our matching
                    layer which mimics this process.
                </p>       
                <table>
    <tbody>
        <tr>
            <td>
                            <img src="./imgs/figure3.png" width="500px" />
            </td>
            <td>
                            <img src="./imgs/figure3_description.png" width="500px" />
            </td>
        </tr>
    </tbody>
</table>
                <p>
                    <strong>Tentative matches in classical geometry estimation.</strong>
                    Classical methods start by computing similarities between
                    all pairs of descriptors across the two images. From this
                    point on, the original descriptors are discarded as all the
                    necessary information for geometry estimation is contained
                    in the pairwise descriptor similarities and their spatial locations. Secondly, the pairs are pruned by either thresholding
                    the similarity values, or, more commonly, only keeping the
                    matches which involve the nearest (most similar) neighbors.
                    Furthermore, the second nearest neighbor test [38] prunes
                    the matches further by requiring that the match strength is
                    significantly stronger than the second best match involving
                    the same descriptor, which is very effective at discarding
                    ambiguous matches.
                </p>
                <p>
                    <strong>Matching layer.</strong> Our matching layer applies a similar procedure. Analogously to the classical approach, only descriptor similarities and their spatial locations should be
                    considered for geometry estimation, and not the original descriptors themselves.
                </p>
                <p>
                    To achieve this, we propose to use a correlation layer
                    followed by normalization. Firstly, all pairs of similarities
                    between descriptors are computed in the correlation layer.
                    Secondly, similarity scores are processed and normalized
                    such that ambiguous matches are strongly down-weighted.
                </p>
                <p>
                    In more detail, given L2-normalized dense feature
                    maps fA, fB ∈ R
                    h×w×d
                    , the correlation map cAB ∈
                    R
                    h×w×(h×w) outputted by the correlation layer contains at
                    each position the scalar product of a pair of individual descriptors fA ∈ fA and fB ∈ fB, as detailed in Eq. (1).
                </p>
                <img src="./imgs/equation1.png" width="500px" />
                <p>
                    where (i, j) and (ik, jk) indicate the individual feature positions in the h×w dense feature maps, and k = h(jk−1)+ik
                    is an auxiliary indexing variable for (ik, jk).
                </p>
                <p>
                    A diagram of the correlation layer is presented in Fig. 3.
                    Note that at a particular position (i, j), the correlation map
                    cAB contains the similarities between fB at that position and
                    all the features of fA.
                </p>
                <p>
                    As is done in the classical methods for tentative correspondence estimation, it is important to postprocess the
                    pairwise similarity scores to remove ambiguous matches.
                    To this end, we apply a channel-wise normalization of the
                    correlation map at each spatial location to produce the final tentative correspondence map fAB. The normalization
                    is performed by ReLU, to zero out negative correlations,
                    followed by L2-normalization, which has two desirable effects. First, let us consider the case when descriptor fB correlates well with only a single feature in fA. In this case,
                    the normalization will amplify the score of the match, akin
                    to the nearest neighbor matching in classical geometry estimation. Second, in the case of the descriptor fB matching
                    multiple features in fA due to the existence of clutter or
                    repetitive patterns, matching scores will be down-weighted
                    similarly to the second nearest neighbor test [38]. However,
                    note that both the correlation and the normalization operations are differentiable with respect to the input descriptors,
                    which facilitates backpropagation thus enabling end-to-end
                    learning.
                </p>
                <p>
                    <strong>Discussion.</strong> The first step of our matching layer, namely
                    the correlation layer, is somewhat similar to layers used in
                    DeepMatching [50] and FlowNet [17]. However, DeepMatching [50] only uses deep RGB patches and no part
                    of their architecture is trainable. FlowNet [17] uses a spatially constrained correlation layer such that similarities are
                    are only computed in a restricted spatial neighborhood thus
                    limiting the range of geometric transformations that can be
                    captured. This is acceptable for their task of learning to estimate optical flow, but is inappropriate for larger transformations that we consider in this work. Furthermore, neither
                    of these methods performs score normalization, which we
                    find to be crucial in dealing with cluttered scenes.
                </p>
                <p>
                    Previous works have used other matching layers to combine descriptors across images, namely simple concatenation of descriptors along the channel dimension [14] or subtraction [29]. However, these approaches suffer from two
                    problems. First, as following layers are typically convolutional, these methods also struggle to handle large transformations as they are unable to detect long-range matches.
                    Second, when concatenating or subtracting descriptors, instead of computing pairwise descriptor similarities as is
                    commonly done in classical geometry estimation and mimicked by the correlation layer, image content information
                    is directly outputted. To further illustrate why this can be
                    problematic, consider two pairs of images that are related
                    with the same geometric transformation – the concatenation
                    and subtraction strategies will produce different outputs for
                    the two cases, making it hard for the regressor to deduce the geometric transformation. In contrast, the correlation layer
                    output is likely to produce similar correlation maps for the
                    two cases, regardless of the image content, thus simplifying the problem for the regressor. In line with this intuition,
                    in Sec. 5.5 we show that the concatenation and subtraction
                    methods indeed have difficulties generalizing beyond the
                    training set, while our correlation layer achieves generalization yielding superior results.
                </p>

### **Regression network**
<p>
                    The normalized correlation map is passed through a regression network which directly estimates parameters of the
                    geometric transformation relating the two input images. In
                    classical geometry estimation, this step consists of robustly
                    estimating the transformation from the list of tentative correspondences. Local geometric constraints are often used to
                    further prune the list of tentative matches [43, 47] by only
                    retaining matches which are consistent with other matches
                    in their spatial neighborhood. Final geometry estimation is
                    done by RANSAC [18] or Hough voting [32, 34, 38].
                </p>
                <p>
                    We again mimic the classical approach using a neural
                    network, where we stack two blocks of convolutional layers, followed by batch normalization [26] and the ReLU
                    non-linearity, and add a final fully connected layer which
                    regresses to the parameters of the transformation, as shown
                    in Fig. 4. The intuition behind this architecture is that the
                    estimation is performed in a bottom-up manner somewhat
                    like Hough voting, where early convolutional layers vote
                    for candidate transformations, and these are then processed
                    by the later layers to aggregate the votes. The first convolutional layers can also enforce local neighborhood consensus
                    [43, 47] by learning filters which only fire if nearby descriptors in image A are matched to nearby descriptors in image
                    B, and we show qualitative evidence in Sec. 5.5 that this indeed does happen.
                </p>
                <p>
                    <strong>Discussion.</strong> A potential alternative to a convolutional regression network is to use fully connected layers. However,
                    as the input correlation map size is quadratic in the number
                    of image features, such a network would be hard to train
                    due to a large number of parameters that would need to be
                    learned, and it would not be scalable due to occupying too
                    much memory and being too slow to use. It should be noted
                    that even though the layers in our architecture are convolutional, the regressor can learn to estimate large transformations. This is because one spatial location in the correlation map contains similarity scores between the corresponding
                    feature in image B and all the features in image A (c.f. equation (1)), and not just the local neighborhood as in [17].
                </p>

### **3.4. Hierarchy of transformations**
<p>
                    Another commonly used approach when estimating image to image transformations is to start by estimating a
                    simple transformation and then progressively increase the
                    model complexity, refining the estimates along the way
                    [11, 37, 40]. The motivation behind this method is that estimating a very complex transformation could be hard and
                    computationally inefficient in the presence of clutter, so a
                    robust and fast rough estimate of a simpler transformation
                    can be used as a starting point, also regularizing the subsequent estimation of the more complex transformation.
                </p>
                <p>
                    We follow the same good practice and start by estimating an affine transformation, which is a 6 degree of freedom
                    linear transformation capable of modeling translation, rotation, non-isotropic scaling and shear. The estimated affine
                    transformation is then used to align image B to image A using an image resampling layer [27]. The aligned images are
                    then passed through a second geometry estimation network
                    which estimates 18 parameters of a thin-plate spline transformation. The final estimate of the geometric transformation is then obtained by composing the two transformations,
                    which is also a thin-plate spline. The process is illustrated
                    in Fig. 5.
                </p>

### **Training**

In order to train the parameters of our geometric matching CNN, it is necessary to design the appropriate loss function, and to use suitable training data. We address these two
                important points next.


### **4.1 Loss function**
<p>        
                    We assume a fully supervised setting, where the training data consists of pairs of images and the desired outputs
                    in the form of the parameters θGT of the ground-truth geometric transformation. The loss function L is designed to
                    compare the estimated transformation ˆθ with the groundtruth transformation θGT and, more importantly, compute
                    the gradient of the loss function with respect to the estimates ∂L
                    ∂θˆ
                    . This gradient is then used in a standard manner
                    to learn the network parameters which minimize the loss
                    function by using backpropagation and Stochastic Gradient
                    Descent.
                </p>
                <img src="./imgs/figure5.png" />
                <p>
                    It is desired for the loss to be general and not specific
                    to a particular type of geometric model, so that it can be
                    used for estimating affine, homography, thin-plate spline or
                    any other geometric transformation. Furthermore, the loss
                    should be independent of the parametrization of the transformation and thus should not directly operate on the parameter values themselves. We address all these design constraints by measuring loss on an imaginary grid of points
                    which is being deformed by the transformation. Namely,
                    we construct a grid of points in image A, transform it using
                    the ground truth and neural network estimated transformations TθGT and Tθˆ with parameters θGT and ˆθ, respectively,
                    and measure the discrepancy between the two transformed
                    grids by summing the squared distances between the corresponding grid points:
                </p>
                <img src="./imgs/equation2.png" width="500px" />
                <p>
                    where G = {gi} = {(xi
                    , yi)} is the uniform grid used,
                    and N = |G|. We define the grid as having xi
                    , yi ∈ {s :
                    s = −1 + 0.1 × n, n ∈ {0, 1, . . . , 20}}, that is to say,
                    each coordinate belongs to a partition of [−1, 1] in equally
                    spaced subintervals of steps 0.1. Note that we construct
                    the coordinate system such that the center of the image is at
                    (0, 0) and that the width and height of the image are equal to
                    2, i.e. the bottom left and top right corners have coordinates
                    (−1, −1) and (1, 1), respectively.
                </p>
                <p>
                    The gradient of the loss function with respect to the
                    transformation parameters, needed to perform backpropagation in order to learn network weights, can be computed
                    easily if the location of the transformed grid points Tθˆ(gi) is
                    differentiable with respect to ˆθ. This is commonly the case,
                    for example, when T is an affine transformation, Tθˆ(gi) is
                    linear in parameters ˆθ and therefore the loss can be differentiated in a straightforward manner.
                </p>

### **4.2. Training from synthetic transformations**

<img src="./imgs/figure6.png" width="500px" />
                <img src="./imgs/figure6_description.png" width="500px" />
                <p>
                    Our training procedure requires fully supervised training
                    data consisting of image pairs and a known geometric relation. Training CNNs usually requires a lot of data, and no
                    public datasets exist that contain many image pairs annotated with their geometric transformation. Therefore, we
                    opt for training from synthetically generated data, which
                    gives us the flexibility to gather as many training examples
                    as needed, for any 2-D geometric transformation of interest.
                    We generate each training pair (IA, IB), by sampling IA from a public image dataset, and generating IB by applying
                    a random transformation TθGT to IA. More precisely, IA
                    is created from the central crop of the original image, while
                    IB is created by transforming the original image with added
                    symmetrical padding in order to avoid border artifacts; the
                    procedure is shown in Fig. 6.
                </p>


### **5. Experimental results**

In this section we describe our datasets, give implementation details, and compare our method to baselines and the
                state-of-the-art. We also provide further insights into the
                components of our architecture.

### **5.1. Evaluation dataset and performance measure**

Quantitative evaluation of our method is performed on
                the Proposal Flow dataset of Ham et al. [22]. The dataset
                contains 900 image pairs depicting different instances of the
                same class, such as ducks and cars, but with large intraclass variations, e.g. the cars are often of different make,
                or the ducks can be of different subspecies. Furthermore,
                the images contain significant background clutter, as can be
                seen in Fig. 8. The task is to predict the locations of predefined keypoints from image A in image B. We do so by
                estimating a geometric transformation that warps image A
                into image B, and applying the same transformation to the
                keypoint locations. We follow the standard evaluation metric used for this benchmark, i.e. the average probability of
                correct keypoint (PCK) [51], being the proportion of keypoints that are correctly matched. A keypoint is considered
                to be matched correctly if its predicted location is within a
                distance of α · max(h, w) of the target keypoint position,
                where α = 0.1 and h and w are the height and width of the
                object bounding box, respectively.

### **5.2. Training data**

<p>
                    Two different training datasets for the affine and
                    thin-plate spline stages, dubbed StreetView-synth-aff and
                    StreetView-synth-tps respectively, were generated by applying synthetic transformations to images from the Tokyo.
                </p>
                <p>
                    Time Machine dataset [5] which contains Google Street
                    View images of Tokyo.
                    Each synthetically generated dataset contains 40k images, divided into 20k for training and 20k for validation.
                    The ground truth transformation parameters were sampled
                    independently from reasonable ranges, e.g. for the affine
                    transformation we sample the relative scale change of up
                    to 2×, while for thin-plate spline we randomly jitter a 3 × 3
                    grid of control points by independently translating each
                    point by up to one quarter of the image size in all directions.
                </p>
                <p>
                    In addition, a second training dataset for the affine stage
                    was generated, created from the training set of Pascal VOC
                    2011 [16] which we dubbed Pascal-synth-aff. In Sec. 5.5,
                    we compare the performance of networks trained with
                    StreetView-synth-aff and Pascal-synth-aff and demonstrate
                    the generalization capabilities of our approach.
                </p>


### **5.3. Implementation details**

<p>
                    We use the MatConvNet library [49] and train the networks with stochastic gradient descent, with learning rate
                    10−3
                    , momentum 0.9, no weight decay and batch size of
                    16. There is no need for jittering as instead of data augmentation we can simply generate more synthetic training
                    data. Input images are resized to 227 × 227 producing
                    15×15 feature maps that are passed into the matching layer.
                    The affine and thin-plate spline stages are trained independently with the StreetView-synth-aff and StreetView-synthtps datasets, respectively. Both stages are trained until convergence which typically occurs after 10 epochs, and takes
                    12 hours on a single GPU. Our final method for estimating
                    affine transformations uses an ensemble of two networks
                    that independently regress the parameters, which are then
                    averaged to produce the final affine estimate. The two networks were trained on different ranges of affine transformations. As in Fig. 5, the estimated affine transformation is
                    used to warp image A and pass it together with image B to a
                    second network which estimates the thin-plate spline transformation. All training and evaluation code, as well as our
                    trained networks, are online at [1].
                </p>

<table>
    <tbody>
        <tr>
            <td>
            <img src="./imgs/table1.png" width="500px" />
            </td>
            <td>
            <img src="./imgs/table1_description.png" width="500px" />
            </td>
        </tr>
    </tbody>
</table>


### **5.4. Comparison to state-of-the-art**

<p>
                    We compare our method against SIFT Flow [35], Graphmatching kernels (GMK) [15], Deformable spatial pyramid
                    matching (DSP) [30], DeepFlow [41], and all three variants
                    of Proposal Flow (NAM, PHM, LOM) [22]. As shown in
                    Tab. 1, our method outperforms all others and sets the new
                    state-of-the-art on this data. The best competing methods
                    are based on Proposal Flow and make use of object proposals, which enables them to guide the matching towards
                    regions of images that contain objects. Their performance
                    varies significantly with the choice of the object proposal
                    method, illustrating the importance of this guided matching. On the contrary, our method does not use any guiding,
                    but it still manages to outperform even the best Proposal
                    Flow and object proposal combination.
                </p>
                <p>
                    Furthermore, we also compare to affine transformations
                    estimated with RANSAC using the same descriptors as our
                    method (VGG-16 pool4). The parameters of this baseline
                    have been tuned extensively to obtain the best result by adjusting the thresholds for the second nearest neighbor test
                    and by pruning proposal transformations which are outside
                    of the range of likely transformations. Our affine estimator
                    outperforms the RANSAC baseline on this task with 49%
                    (ours) compared to 47% (RANSAC).
                </p>

### **5.5. Discussions and ablation studies**

<p>        
                    In this section we examine the importance of various
                    components of our architecture. Apart from training on the
                    StreetView-synth-aff dataset, we also train on Pascal-synthaff which contains images that are more similar in nature to
                    the images in the Proposal Flow benchmark. The results of
                    these ablation studies are summarized in Tab. 2.
                </p>
                <img src="./imgs/figure7.png" />
                <img src="./imgs/figure8.png" />
                <img src="./imgs/table2.png" width="500px" />
                <img src="./imgs/table2_description.png" width="500px" />
                <p>
                    <strong>Correlation versus concatenation and subtraction.</strong> Replacing our correlation-based matching layer with feature
                    concatenation or subtraction, as proposed in [14] and [29], respectively, incurs a large performance drop. The behavior
                    is expected as we designed the matching layer to only keep
                    information on pairwise descriptor similarities rather than
                    the descriptors themselves, as is good practice in classical
                    geometry estimation methods, while concatenation and subtraction do not follow this principle.
                </p>
                <p>
                    <strong>Generalization.</strong> As seen in Tab. 2, our method is relatively
                    unaffected by the choice of training data as its performance
                    is similar regardless whether it was trained with StreetView
                    or Pascal images. We also attribute this to the design choice
                    of operating on pairwise descriptor similarities rather than
                    the raw descriptors.
                </p>
                <img src="./imgs/figure9.png" />
                <p>
                    <strong>Normalization.</strong> Tab. 2 also shows the importance of the
                    correlation map normalization step, where the normalization improves results from 44% to 49%. The step mimics
                    the second nearest neighbor test used in classical feature
                    matching [38], as discussed in Sec. 3.2. Note that [17] also
                    uses a correlation layer, but they do not normalize the map
                    in any way, which is clearly suboptimal.
                </p>
                <p>
                    <strong>What is being learned?</strong> We examine filters from the first
                    convolutional layer of the regressor, which operate directly
                    on the output of the matching layer, i.e. the tentative correspondence map. Recall that each spatial location in the correspondence map (see Fig. 3, in green) contains all similarity scores between that feature in image B and all features in
                    image A. Thus, each single 1-D slice through the weights of
                    one convolutional filter at a particular spatial location can be
                    visualized as an image, showing filter’s preferences to features in image B that match to specific locations in image A.
                    For example, if the central slice of a filter contains all zeros
                    apart from a peak at the top-left corner, this filter responds
                    positively to features in image B that match to the top-left
                    of image A. Similarly, if many spatial locations of the filter produce similar visualizations, then this filter is highly
                    sensitive to spatially co-located features in image B that all
                    match to the top-left of image A. For visualization, we pick
                    the peaks from all slices of filter weights and average them
                    together to produce a single image. Several filters shown
                    in Fig. 7 confirm our hypothesis that this layer has learned
                    to mimic local neighborhood consensus as some filters respond strongly to spatially co-located features in image B
                    that match to spatially consistent locations in image A. Furthermore, it can be observed that the size of the preferred
                    spatial neighborhood varies across filters, thus showing that
                    the filters are discriminative of the scale change.
                </p>

### **5.6. Qualitative results**
     
<p>
Fig. 8 illustrates the effectiveness of our method in
category-level matching, where challenging pairs of images
from the Proposal Flow dataset [22], containing large intraclass variations, are aligned correctly. The method is able
to robustly, in the presence of clutter, estimate large translations, rotations, scale changes, as well as non-rigid transformations and some perspective changes. Further examples
are shown in the supplementary material [2] .
</p>
<p>
Fig. 9 shows the quality of instance-level matching,
where different images of the same scene are aligned correctly. The images are taken from the Tokyo Time Machine
dataset [5] and are captured at different points in time which
are months or years apart. Note that, by automatically highlighting the differences (in the feature space) between the
aligned images, it is possible to detect changes in the scene,
such as occlusions, changes in vegetation, or structural differences e.g. new buildings being built.
</p>


### **6. Conclusions**

<p>
            We have described a network architecture for geometric matching fully trainable from synthetic imagery without the need for manual annotations. Thanks to our matching layer, the network generalizes well to never seen before imagery, reaching state-of-the-art results on the challenging Proposal Flow dataset for category-level matching.
This opens-up the possibility of applying our architecture
to other difficult correspondence problems such as matching across large changes in illumination (day/night) [5] or
depiction style [6].
</p>

<p>
<strong>Acknowledgements.</strong> This work has been partly supported
by ERC grant LEAP (no. 336845), ANR project Semapolis (ANR-13-CORD-0003), the Inria CityLab IPL, CIFAR
Learning in Machines & Brains program and ESIF, OP
Research, development and education Project IMPACT No.
CZ.02.1.01/0.0/0.0/15 003/0000468.
</p>

### **References**

- [1] Project webpage (code/networks). http://www.di.
ens.fr/willow/research/cnngeometric/.
- [2] Supplementary material (appendix) for the paper. https:
//arxiv.org/abs/1703.05593.
- [3] S. Agarwal, N. Snavely, I. Simon, S. M. Seitz, and
R. Szeliski. Building Rome in a day. In Proc. ICCV, 2009.
- [4] H. Altwaijry, E. Trulls, J. Hays, P. Fua, and S. Belongie.
Learning to match aerial images with deep attentive architectures. In Proc. CVPR, 2016.
- [5] R. Arandjelovic, P. Gronat, A. Torii, T. Pajdla, and J. Sivic. ´
NetVLAD: CNN architecture for weakly supervised place
recognition. In Proc. CVPR, 2016.
- [6] M. Aubry, B. Russell, and J. Sivic. Painting-to-3D model
alignment via discriminative visual elements. ACM Transactions on Graphics, 2013.
- [7] H. Azizpour, A. Razavian, J. Sullivan, A. Maki, and S. Carlsson. Factors of transferability from a generic ConvNet representation. arXiv preprint arXiv:1406.5774, 2014.
- [8] A. Babenko and V. Lempitsky. Aggregating local deep features for image retrieval. In Proc. ICCV, 2015.
- [9] V. Balntas, E. Johns, L. Tang, and K. Mikolajczyk. PN-Net:
Conjoined triple deep network for learning local image descriptors. arXiv preprint arXiv:1601.05030, 2016.
- [10] H. Bay, T. Tuytelaars, and L. Van Gool. Surf: Speeded up
robust features. In Proc. ECCV, 2006.
- [11] A. Berg, T. Berg, and J. Malik. Shape matching and object
recognition using low distortion correspondence. In Proc.
CVPR, 2005.
- [12] N. Dalal and B. Triggs. Histogram of Oriented Gradients for
Human Detection. In Proc. CVPR, 2005.
- [13] J. Deng, W. Dong, R. Socher, L.-J. Li, K. Li, and L. FeiFei. ImageNet: A large-scale hierarchical image database.
In Proc. CVPR, 2009.
- [14] D. DeTone, T. Malisiewicz, and A. Rabinovich. Deep image
homography estimation. arXiv preprint arXiv:1606.03798, 2016.
- [15] O. Duchenne, A. Joulin, and J. Ponce. A graph-matching
kernel for object categorization. In Proc. ICCV, 2011.
- [16] M. Everingham, L. Van Gool, C. K. I. Williams, J. Winn,
and A. Zisserman. The PASCAL Visual Object Classes
Challenge 2011 (VOC2011) Results. http://www.pascalnetwork.org/challenges/VOC/voc2011/workshop/index.html.
- [17] P. Fischer, A. Dosovitskiy, E. Ilg, P. Hausser, C. Hazırbas¸, ¨
V. Golkov, P. van der Smagt, D. Cremers, and T. Brox.
FlowNet: Learning optical flow with convolutional networks. In Proc. ICCV, 2015.
- [18] M. A. Fischler and R. C. Bolles. Random sample consensus: A paradigm for model fitting with applications to image
analysis and automated cartography. Comm. ACM, 1981.
- [19] D. A. Forsyth and J. Ponce. Computer vision: a modern
approach. Prentice Hall Professional Technical Reference, 2002.
- [20] Y. Gong, L. Wang, R. Guo, and S. Lazebnik. Multi-scale
orderless pooling of deep convolutional activation features.
In Proc. ECCV, 2014.
- [21] Y. HaCohen, E. Shechtman, D. B. Goldman, and D. Lischinski. Non-rigid dense correspondence with applications for
image enhancement. Proc. ACM SIGGRAPH, 2011.
- [22] B. Ham, M. Cho, C. Schmid, and J. Ponce. Proposal Flow.
In Proc. CVPR, 2016.
- [23] X. Han, T. Leung, Y. Jia, R. Sukthankar, and A. C. Berg.
MatchNet: Unifying feature and metric learning for patchbased matching. In Proc. CVPR, 2015.
- [24] C. Harris and M. Stephens. A combined corner and edge
detector. In Alvey vision conference, 1988.
- [25] R. Hartley and A. Zisserman. Multiple view geometry in
computer vision. Cambridge university press, 2003.
- [26] S. Ioffe and C. Szegedy. Batch Normalization: Accelerating
deep network training by reducing internal covariate shift. In
Proc. ICML, 2015.
- [27] M. Jaderberg, K. Simonyan, A. Zisserman, and
K. Kavukcuoglu. ”Spatial Transformer Networks”. In
NIPS, 2015.
- [28] M. Jahrer, M. Grabner, and H. Bischof. Learned local descriptors for recognition and matching. In Computer Vision
Winter Workshop, 2008.
- [29] A. Kanazawa, D. W. Jacobs, and M. Chandraker. WarpNet:
Weakly supervised matching for single-view reconstruction.
In Proc. CVPR, 2016.
- [30] J. Kim, C. Liu, F. Sha, and K. Grauman. Deformable spatial
pyramid matching for fast dense correspondences. In Proc.
CVPR, 2013.
- [31] A. Krizhevsky, I. Sutskever, and G. E. Hinton. ImageNet
classification with deep convolutional neural networks. In
NIPS, 2012.
- [32] Y. Lamdan, J. T. Schwartz, and H. J. Wolfson. Object recognition by affine invariant matching. In Proc. CVPR, 1988.
- [33] E. G. Learned-Miller. Data driven image models through
continuous joint alignment. IEEE PAMI, 2006.
- [34] B. Leibe, A. Leonardis, and B. Schiele. Robust object detection with interleaved categorization and segmentation. IJCV, 2008.
- [35] C. Liu, J. Yuen, and A. Torralba. SIFT Flow: Dense correspondence across scenes and its applications. IEEE PAMI, 2011.
- [36] J. L. Long, N. Zhang, and T. Darrell. Do convnets learn
correspondence? In NIPS, 2014.
- [37] D. G. Lowe. Object recognition from local scale-invariant
features. In Proc. ICCV, 1999.
- [38] D. G. Lowe. Distinctive image features from scale-invariant
keypoints. IJCV, 2004.
- [39] K. Mikolajczyk and C. Schmid. An affine invariant interest
point detector. In Proc. ECCV, 2002.
- [40] J. Philbin, O. Chum, M. Isard, J. Sivic, and A. Zisserman. Object retrieval with large vocabularies and fast spatial
matching. In Proc. CVPR, 2007.
- [41] J. Revaud, P. Weinzaepfel, Z. Harchaoui, and C. Schmid.
DeepMatching: Hierarchical deformable dense matching.
IJCV, 2015.
- [42] M. Rubinstein, A. Joulin, J. Kopf, and C. Liu. Unsupervised
joint object discovery and segmentation in internet images.
In Proc. CVPR, 2013.
- [43] C. Schmid and R. Mohr. Local grayvalue invariants for image retrieval. IEEE PAMI, 1997.
- [44] F. Shokrollahi Yancheshmeh, K. Chen, and J.-K. Kamarainen. Unsupervised visual alignment with similarity
graphs. In Proc. CVPR, 2015.
- [45] E. Simo-Serra, E. Trulls, L. Ferraz, I. Kokkinos, P. Fua, and 6156
F. Moreno-Noguer. Discriminative learning of deep convolutional feature point descriptors. In Proc. ICCV, 2015.
- [46] K. Simonyan and A. Zisserman. Very deep convolutional
networks for large-scale image recognition. In Proc. ICLR, 2015.
- [47] J. Sivic and A. Zisserman. Video Google: A text retrieval
approach to object matching in videos. In Proc. ICCV, 2003.
- [48] J. Thewlis, S. Zheng, P. Torr, and A. Vedaldi. Fully-trainable
deep matching. In Proc. BMVC., 2016.
- [49] A. Vedaldi and K. Lenc. MatConvNet – Convolutional neural
networks for MATLAB. In Proc. ACMM, 2015.
- [50] P. Weinzaepfel, J. Revaud, Z. Harchaoui, and C. Schmid.
DeepFlow: Large displacement optical flow with deep
matching. In Proc. ICCV, 2013.
- [51] Y. Yang and D. Ramanan. Articulated human detection with
flexible mixtures of parts. IEEE PAMI, 2013.
- [52] S. Zagoruyko and N. Komodakis. Learning to compare image patches via convolutional neural networks. In Proc.
CVPR, 2015.
- [53] T. Zhou, Y. J. Lee, S. X. Yu, and A. A. Efros. ”FlowWeb:
Joint image set alignment by weaving consistent, pixel-wise
correspondences”. In Proc. CVPR, 2015.
